## Actuator Control Allocation Example

When controlling physical systems like spacecraft or automobiles, the controller commands a desired control input $w \in \mathbb{R}^m$, which might be a wrench vector, containg forces and torques ($m=6$).

Usually, multiple atuators are available to produce this control input. The vector $u \in \mathbb{R}^n$ contains the single actuator input values in its components. If $n > m$ and the actuators are in general position, we call the system "over-actuated", since there are many realizations of $u$ that result in the same value of $w$ via the linear mapping $A$. 

Having this freedom of choice, we want to minimize energy consumption, modeled as $\kappa^T | u |$ (with $\kappa \geq 0$), while discouraging rapid changes of the actuation values, i.e., $\lambda^\mathrm{sm} \Vert u-u^\mathrm{prev} \Vert_2^2$ with $\lambda^\mathrm{sm} \geq 0$ and $u^\mathrm{prev}$ being the actuation of the previous time step. Given the bounds $u^\mathrm{min}$ and $u^\mathrm{max}$ $(u^\mathrm{min} \leq u^\mathrm{max})$ on $u$, there might be cases when the desired control input $w$ is infeasible. Hence, we only softly penalize deviations between desired and actual control input with the cost term $\Vert A u - w \Vert_2^2$. The resulting optimization problem is the following:

\begin{equation}
\begin{array}{II}
\text{minimize} \quad &\Vert A u - w \Vert_2^2  + \lambda^\mathrm{sm} \Vert u-u^\mathrm{prev} \Vert_2^2 + \kappa^T | u |\\
\text{subject to} \quad &u^\mathrm{min} \leq u \leq u^\mathrm{max}
\end{array}
\end{equation}

We introduce the variable $\Delta u = u - u^\mathrm{prev}$ to make the problem [DPP-compliant](https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming):

\begin{equation}
\begin{array}{II}
\text{minimize} \quad &\Vert A u - w \Vert_2^2  + \lambda^\mathrm{sm} \Vert \Delta u \Vert_2^2 + \kappa^T | u |\\
\text{subject to} \quad &u^\mathrm{min} \leq u \leq u^\mathrm{max} \\
&\Delta u = u-u^\mathrm{prev}
\end{array}
\end{equation}

Let's define the corresponding CVXPY problem:

In [13]:
import cvxpy as cp

# define dimensions
n, m = 3, 4

# define variables
u = cp.Variable(n, name='u')
delta_u = cp.Variable(n, name='delta_u')

# define parameters
A = cp.Parameter((m, n), name='A')
w = cp.Parameter(m, name='w')
lamb_sm = cp.Parameter(nonneg=True, name='lamb_sm')
kappa = cp.Parameter(n, nonneg=True, name='kappa')
u_prev = cp.Parameter(n, name='u_prev')
u_min = cp.Parameter(n, name='u_min')
u_max = cp.Parameter(n, name='u_max')

# define objective
objective = cp.Minimize(cp.sum_squares(A@u-w) + lamb_sm*cp.sum_squares(delta_u) + kappa@cp.abs(u))

# define constraints
constraints = [u_min <= u, u <= u_max, delta_u == u-u_prev]

# define problem
problem = cp.Problem(objective, constraints)

Assign parameter values and solve the problem.

In [ ]:
# TODO

val = problem.solve()

Generating C source for the problem is as easy as:

In [ ]:
import sys
sys.path.append('../')
import cvxpygen as cpg

cpg.generate_code(problem, code_dir='actuator_code')

Now, you can use a python wrapper around the generated code as a custom CVXPY solve method:

In [ ]:
from actuator_code.cpg_solver import cpg_solve
import numpy as np
import pickle
import time

# load the serialized problem formulation
with open('actuator_code/problem.pickle', 'rb') as f:
    prob = pickle.load(f)

# assign parameter values
# TODO

# solve problem conventionally
t0 = time.time()
# CVXPY chooses eps_abs=eps_rel=1e-5, max_iter=10000, polish=True by default,
# however, we choose the OSQP default values here, as they are used for code generation as well
val = prob.solve(eps_abs=1e-3, eps_rel=1e-3, max_iter=4000, polish=False)
t1 = time.time()
print('\nPython solve time:', 1000*(t1-t0), 'ms')
print('Python objective function value:', val)

# solve problem with C code via python wrapper
prob.register_solve('CPG', cpg_solve)
t0 = time.time()
val = prob.solve(method='CPG')
t1 = time.time()
print('\nC solve time:', 1000*(t1-t0), 'ms')
print('C objective function value:', val)